In [1]:
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt
import microatoll_sim.simulator as sim

## Setup


In [10]:
radius = 0.30
d = 1.0
gr = 15
dt = 0.1
Tmax = 30
T0 = 1980
vert_shift = 0.4
num_initial_pts = 200

## post processing
Tmax *= 1.00001
d = d / 1e3
gr = (gr / 1e3) * dt
NT = int(Tmax / dt)

sl_df = pd.read_csv("./data/SeaLevel.csv", header=None, names=["time", "sl"])
sl_arr = sl_df.to_numpy()
sl_arr[:, 1] += vert_shift
band_sl_arr = sim.lowest_discreet(sl_arr, dt, T0, T0 + Tmax)
gr_vec = np.ones(NT) * gr

## Core simulator benchmarks


In [4]:
%timeit sim.coral_growth(radius, num_initial_pts, d, gr_vec, NT, band_sl_arr[:, 1])

164 ms ± 392 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
%timeit sim.lowest_discreet(sl_arr, dt, T0, T0 + Tmax)

78.4 µs ± 240 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [6]:
%timeit sim.resample(lines, living_statuses,d)

21.7 µs ± 4.44 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%timeit sim.kill_loop(lines, living_statuses,gr) 

2.15 ms ± 17.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
%timeit sim.get_pointwise_unit_normals(lines)

3.49 µs ± 17.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
